# Release and Deployment

After stepping through these development iterations and achieving **sufficient performance** and successful **test results**, we can proceed to releasing the particular version of the solution model and deploying it for inference.

In [1]:
%cd forml-solution-avazuctr

/opt/forml/workspace/3-solution/forml-solution-avazuctr


## Releasing

Let's release the first version of our model. As per the `version` field in the [pyproject.toml](forml-solution-avazuctr), the version number is going to be `0.1`.

Following the best practices, we first commit all the recent changes and tag it with the version number:

In [2]:
! git commit -m "Released 0.1"
! git tag "v0.1"

[main (root-commit) 096ff6a] Released 0.1
 9 files changed, 200 insertions(+)
 create mode 100644 .gitignore
 create mode 100644 avazuctr/__init__.py
 create mode 100644 avazuctr/evaluation.py
 create mode 100644 avazuctr/pipeline.py
 create mode 100644 avazuctr/source.py
 create mode 100644 pyproject.toml
 create mode 100644 tests/__init__.py
 create mode 100644 tests/test_pipeline.py
 create mode 100644 tests/test_source.py


### Packaging & Publishing

Now we can trigger the actual packaging of the artifact and publish it to the model registry:

In [3]:
! forml project release

running bdist_4ml
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 910.0 kB/s eta 0:00:00a 0:00:01
  Using cached forml-0.93-py3-none-any.whl (283 kB)
  Using cached imbalanced_learn-0.10.1-py3-none-any.whl (226 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 869.3 kB/s eta 0:00:0000:0100:01
  Using cached scikit_learn-1.2.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.6 MB)
  Using cached numpy-1.24.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Using cached scipy-1.10.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.4 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 880.9 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.8/233.8 kB 974.1 kB/s eta 0:00:00a 0:00:01
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Using cached cloudpickle-2.2.1-py3-none-any.whl (25 kB)
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached packaging-23.1-py3-none-any.

### Browsing the Model Registry

We should now see the project in the model registry next to the previously published `dummy`:

In [4]:
! forml model list

dummy                    forml-solution-avazuctr  


In [5]:
! forml model list forml-solution-avazuctr

0.1  


In [6]:
! forml model list forml-solution-avazuctr 0.1

The release `0.1` is now in the model registry but it has no trained _generations_ yet.

In [7]:
! tree /opt/forml/assets/registry/forml-solution-avazuctr

/opt/forml/assets/registry/forml-solution-avazuctr
└── 0.1
    └── package.4ml

1 directory, 1 file


### Training

Let's pretend the current time is `2014-10-21 03:00:00` and train a model with data up to that _now_ point:

In [8]:
! forml model train forml-solution-avazuctr \
    --upper '2014-10-21 03:00:00' 

In [9]:
! forml model list forml-solution-avazuctr 0.1

1  


In [10]:
! tree /opt/forml/assets/registry/forml-solution-avazuctr

/opt/forml/assets/registry/forml-solution-avazuctr
└── 0.1
    ├── 1
    │   ├── 0444977f-94e0-41af-95cc-bc73c7d4190e.bin
    │   ├── 13459cdb-6c85-4899-a516-f814d5b2bda2.bin
    │   ├── 32e05554-5eed-44dd-b53d-23b8807562a3.bin
    │   ├── 38442d05-75ce-4a00-954e-aef27cacd4e7.bin
    │   ├── 4cb2c4c6-7cd2-47b6-a37d-daf15119e46f.bin
    │   ├── 66ab3016-2baa-4864-915f-13ab9163ce60.bin
    │   ├── 83ed1abf-4086-4d05-9ad4-4e9f086c5d68.bin
    │   ├── 8b94be64-ea35-4642-b26a-f98a6317be20.bin
    │   ├── 8c736b7e-6fc6-4918-815a-4badc0d63af8.bin
    │   └── tag.toml
    └── package.4ml

2 directories, 11 files


## Deployment & Serving

Serving our published model means exposing its task graph in the _apply mode_ for prediction requests.

To quickly remind ourselves what this task graph looks like:

In [11]:
! forml model -R graphviz apply forml-solution-avazuctr

_forml-solution-avazuctr/forml.dot.svg_:
[![Apply Task Graph](img/apply.svg)](img/apply.svg)

### Serving Gateways

On top of the published models, ForML adds an additional layer for controlling all sorts of model serving aspects. This brings forth the concept of [applications](https://docs.forml.io/en/latest/application.html) and [serving gateways](https://docs.forml.io/en/latest/serving.html).

Let's start the default serving gateway. Since it is a server process, launch it in a **separate terminal** using the following command:

```shell
$ forml application serve
```

We can now issue a request to the gateway targetting the (not-yet-deployed) app `forml-solution-avazuctr`:

In [12]:
! curl -X POST http://127.0.0.1:8000/forml-solution-avazuctr

Application forml-solution-avazuctr not found in Dispatch-registry

As expected, this results in `404` until the application is actually deployed.

### Application

Applications are defined through an _application descriptor_ responsible for:
* **decoding and extracting** the prediction request
* **selecting a model** for serving the prediction request
* **assembling and encoding** the prediction response

Let's implement a simple application descriptor for our scenario:

In [13]:
! touch application.py

Now, add the generic application descriptor code to the [application.py](forml-solution-avazuctr/application.py):

1. Open the [application.py](forml-solution-avazuctr/application.py) component.
2. Update it with the code below registering the generic application descriptor:
```python
from forml import application

application.setup(application.Generic('forml-solution-avazuctr'))
```
3. **SAVE THE [application.py](forml-solution-avazuctr/application.py) FILE!**

In [14]:
! git add application.py

The application gets deployed by pushing its descriptor to the _application inventory_:

In [15]:
! forml application put application.py
! forml application list

forml-solution-avazuctr  


Now we can issue a prediction request:

In [16]:
! curl -H 'Content-Type: application/json' -d '[{ \
    "hour": "2014-10-21 03:00:00", \
    "banner_pos": "0", \
    "site_id": "887a4754", "site_domain": "e3d9ca35", \
    "site_category": "50e219e0", \
    "app_id": "ecad2386", "app_domain": "7801e8d9", \
    "app_category": "07d7df22", \
    "device_id": "0e79d423", "device_ip": "9f423918", \
    "device_model": "fc10a0d3", \
    "device_type": "0", "device_conn_type": "0", \
    "C1": "1002", "C14": "22701", "C15": "320", "C16": "50", \
    "C17": "2624", "C18": "0", "C19": "35", "C20": "-1", "C21": "221" \
}]' http://127.0.0.1:8000/forml-solution-avazuctr

[{"c0":0.1618960089}]